In [1]:
# !pip install trl
#

In [2]:
# pip install -U bitsandbytes

#main loop

In [ ]:
from huggingface_hub import login

login()


In [ ]:
import os
import torch
from huggingface_hub import InferenceClient
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer


my_token = os.environ.get("api key")
# --- Model and Tokenizer Setup ---
model_name = "meta-llama/Llama-3.2-3B-Instruct"

# --- Corrected Quantization Config ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # <-- 2. Corrected this line
)

# --- Corrected Model Loading ---
tokenizer = AutoTokenizer.from_pretrained(model_name,token=my_token)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # <-- 3. Pass the config here
    device_map="auto",              # <-- 4. Add device_map
    token=my_token
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import pandas as pd

data=pd.read_csv("/content/final.csv")
data.head()

,date,likes,content,username,inferred company,media_url,media_type,generated_caption
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,tim hortons,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,photo,The image is a promotional advertisement for T...
1,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,independent,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,photo,The image shows a person standing in front of ...
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,cbc,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,photo,The image depicts a military scene where a sol...
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,williams,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,photo,The image depicts a beach scene with several d...
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,independent,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,photo,The image shows two individuals standing in wh...


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               2000 non-null   object
 1   likes              2000 non-null   int64 
 2   content            2000 non-null   object
 3   username           2000 non-null   object
 4   inferred company   2000 non-null   object
 5   media_url          2000 non-null   object
 6   media_type         2000 non-null   object
 7   generated_caption  2000 non-null   object
dtypes: int64(1), object(7)
memory usage: 125.1+ KB


In [12]:
import pandas as pd

def make_prompt(row):
    """
    Build a text-to-text training prompt for Task-2 (content simulation).
    Input: row is a dict or pandas Series with keys:
      - date
      - likes
      - username
      - media
      - inferred_company
      - media_caption (optional)

    Output: string prompt
    """

    date = str(row.get("date", ""))
    likes = str(row.get("likes", ""))
    username = str(row.get("username", ""))
    media_type = str(row.get("media_type", ""))
    # Checks for "inferred company" or "inferred_company"
    company = str(row.get("inferred company", row.get("inferred_company", "")))
    caption = str(row.get("generated_caption", ""))

    prompt = f"""
You are a content-generation model trained to simulate Twitter marketing posts.

Your task:
Given structured metadata input, generate realistic marketing tweet text.

Rules:
1. Marketing tone under 280 characters.
2. Align with brand identity from username and company and time.
3. Reference media_caption if present.
4. Do not mention likes explicitly.
5. No explanation. Output tweet text only.

Input:
<date> {date}
<likes> {likes}
<username> {username}
<company> {company}
<media> {media_type}
<media caption> {caption}

Output (content):
"""
    return "\n".join(line.strip() for line in prompt.strip().split("\n"))

try:
    df = pd.read_csv('/content/final.csv')
    df['prompt'] = df.apply(make_prompt, axis=1)

    print("DataFrame with new 'prompt' column:")
    print(df.head())

    output_filename = 'output_with_prompts.csv'
    df.to_csv(output_filename, index=False)

    print(f"\nSuccessfully processed file and saved results to {output_filename}")

except FileNotFoundError:
    print("Error: 'your_input_file.csv' not found.")
    print("Please make sure the file is in the same directory or provide the full path.")
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check your CSV file's column names. They must match what the function expects:")
    print("Expected keys: 'date', 'likes', 'username', 'media_type', 'inferred company' (or 'inferred_company')")

DataFrame with new 'prompt' column:
                  date  likes  \
0  2020-12-12 00:47:00      1   
1  2018-06-30 10:04:20   2750   
2  2020-09-29 19:47:28     57   
3  2020-10-01 11:40:09    152   
4  2018-10-19 14:30:46     41   

                                             content        username  \
0  Spend your weekend morning with a Ham, Egg, an...    TimHortonsPH   
1  Watch rapper <mention> freestyle for over an H...       IndyMusic   
2  Canadian Armenian community demands ban on mil...       CBCCanada   
3  1st in Europe to be devastated by COVID-19, It...  MKWilliamsRome   
4  Congratulations to Pauletha Butts of <mention>...           BGISD   

  inferred company                                          media_url  \
0      tim hortons  https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...   
1      independent  https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...   
2              cbc  https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...   
3         williams  https://pbs.twimg.co

In [13]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

# The pre-trained model you want to fine-tune
model_name = "meta-llama/Llama-3.1-8B-Instruct"

dataset_file = "/content/output_with_prompts.csv"

output_dir = "/content/drive/MyDrive/llama-3.1-8b-custom-adapter"

dataset = load_dataset("csv", data_files=dataset_file, split="train")

def format_data_for_llama3(row):
    """
    Wraps the prompt and content in the Llama 3 chat template.
    We create a list of messages and let the tokenizer handle the
    special tokens (e.g., <|begin_of_text|>, <|eot_id|>, etc.)
    """
    messages = [
        {"role": "user", "content": row['prompt']},
        {"role": "assistant", "content": row['content']}
    ]

    formatted_string = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": formatted_string}

print("Formatting dataset...")
formatted_dataset = dataset.map(
    format_data_for_llama3,
    remove_columns=dataset.column_names
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

lr_scheduler_type = "cosine"
optim = "paged_adamw_32bit"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=6,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
)


print("Initializing trainer...")
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    args=training_args,
)



Generating train split: 0 examples [00:00, ? examples/s]

Formatting dataset...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Initializing trainer...


Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
# # --- 7. Train and Save ---

# print("Starting training...")
# trainer.train()
# print("Training complete!")

# print(f"Saving adapter to {output_dir}")
# trainer.save_model(output_dir)
# print("Done.")

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainer.train(resume_from_checkpoint=True)